In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, RocCurveDisplay

In [14]:
df = pd.read_csv("../Oblig1/wdbc.data", header=None)
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189


In [15]:
df.describe()

,0,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


Ut i fra beskrivelsen til datasetet så er kollone 0 en id verdi, kollone 1 er target verdien med M for malignet og B for benine og de resterende kollonene er features og ut i fra en enkel analyse av datasetet ser disse ut til å hvere normalfordelte tallverdier.

In [3]:
df["target"] = np.where(df[1] == "M", 1, 0)
df = df.drop([0,1], axis=1)

Før vi bruker datasete videre konvertere vi verdiene i target kollonen til 1 for M og 0 for B, slik at disse er på et format som kan brukes senere. Videre legger vi disse verdiene i en ny kollone "target", slik at vi letter kan bruke generelle funksjoner for splitting av datasetet. Siden verdiene for ID kollonen og den gamle kollonen for target verdiene ikke har noe videre nytteverdi, så dropper vi disse dra datasetet.

ref for np.where: https://www.geeksforgeeks.org/numpy/numpy-where-in-python/

In [4]:
# funksjon for splitting av dataset i train, validate og test
def split_dataset(data):
    train_val = data.groupby("target", group_keys=False)[data.columns].sample(frac=0.8)
    test = data.drop(train_val.index)
    train = train_val.groupby("target", group_keys=False)[train_val.columns].sample(frac=0.75)
    validation = train_val.drop(train.index)    
    return [train, validation, test]

# hjelpe funksjon for å validere av jevn fordeling av dataset på target verdi
def check_split_dataset(data):
    print(data[0]["target"].value_counts()/data[0].shape[0])
    print(data[1]["target"].value_counts()/data[1].shape[0])
    print(data[2]["target"].value_counts()/data[2].shape[0])

# funksjon for å dele opp i targets og features
def split_targets_and_features(data):
    targets = data.target
    features = data.drop("target", axis=1)
    return [targets, features]

# hjelpe funksjon for oppdeling i features og targets
def create_set_of_targets_and_features(data):
    train = split_targets_and_features(data[0])
    validation = split_targets_and_features(data[1])
    test = split_targets_and_features(data[2])

    return [train, validation, test]

# funksjon for spliting av dataset
def create_full_dataset(input):
    data = split_dataset(input)
    dataset = create_set_of_targets_and_features(data)

    return dataset

I seksjonen over så ligger kode for alle fuksjonene som brukes til å splitte opp datasettet.
Jeg har valgt å bruke en 60, 20, 20 % split, slik at jeg får en god fordeling mellom dataene for trening, validering og testing. Ut ifra størelsen på dette datasette mener jeg dette gir en grei balanse mellom nok data til trening av moddelen, men samtidig en god mengde data til opptimalisering og testing av moddelen. Hvis datasetet hadde vært av betydelig mindre størelse kunne det vært lurt å tildele en større andel av det totale datasetet til trening av modellen for å få en moddel som kan høyde for en større spredning av tilfeller.

In [16]:
# hjelpe fuksjon for vurdering av modellene        
def evaluate_model(classifier, features, targets):
    predictions = classifier.predict(features)
    accuracy = accuracy_score(targets, predictions)
    precision = precision_score(targets, predictions)
    recall = recall_score(targets, predictions)
    f1 = f1_score(targets, predictions)
    roc_auc = roc_auc_score(targets, predictions)   

    return [accuracy, precision, recall, f1, roc_auc]

# hjelpe funksjon for å legg til verdier i resultat array i validate og test funksjonen 
def append_values(results, accuracy, precision, recall, f1, roc_auc):
        accuracy.append(results[0])
        precision.append(results[1])
        recall.append(results[2])
        f1.append(results[3])
        roc_auc.append(results[4])

# hjelpe funksjon for å printe resultatene fra validate funksjonen
def results_print(accuracy, precision, recall, f1, roc_auc):
    print(f'Accuracy: {np.mean(accuracy):.2f}')
    print(f'Precision: {np.mean(precision):.2f}')
    print(f'Recall: {np.mean(recall):.2f}')
    print(f'F1: {np.mean(f1):.2f}')
    print(f'ROCAUC: {np.mean(roc_auc):.2f}')

# hjelpe funksjon for å printe resultatene fra test funksjonen
def test_results_print(accuracy, precision, recall, f1, roc_auc):
    print(f'Accuracy: {np.mean(accuracy):.2f} +- {np.std(accuracy):.2f}')
    print(f'Precision: {np.mean(precision):.2f} +- {np.std(precision):.2f}')
    print(f'Recall: {np.mean(recall):.2f} +- {np.std(recall):.2f}')
    print(f'F1: {np.mean(f1):.2f} +- {np.std(f1):.2f}')
    print(f'ROCAUC: {np.mean(roc_auc):.2f} +- {np.std(roc_auc):.2f}')

# funksjon for å validere modellene og hyperparameter tuningen
def validate(classifier, runs):    
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = [], [], [], [], []
    validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc = [], [], [], [], []

    for i in range(0, runs, 1):
        dataset = create_full_dataset(df)
        classifier.fit(dataset[0][1], dataset[0][0])
    
        results_train = evaluate_model(classifier, dataset[0][1], dataset[0][0])
        append_values(results_train, train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
    
        results_validate = evaluate_model(classifier, dataset[1][1], dataset[1][0])
        append_values(results_validate, validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc)

    print('Training results:')
    results_print(train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
    print('Validation results:')
    results_print(validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc)

# funksjon for endelig testing av modellene og hyperparameter tuningen
def test(classifier, runs):
    accuracy, precision, recall, f1, roc_auc = [], [], [], [], []

    for i in range(0, runs, 1):
        dataset = create_full_dataset(df)
        classifier.fit(dataset[0][1], dataset[0][0])

        results = evaluate_model(classifier, dataset[2][1], dataset[2][0])
        append_values(results, accuracy, precision, recall, f1, roc_auc)
    
    print('Test results:')
    test_results_print(accuracy, precision, recall, f1, roc_auc)


Kort kommentar om funksjoner for trening og testing

In [ ]:

dt1 = DecisionTreeClassifier()
validate(dt1, 20)



Training results:
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1: 1.00
ROCAUC: 1.00
Validation results:
Accuracy: 0.93
Precision: 0.90
Recall: 0.91
F1: 0.90
ROCAUC: 0.92
Test results:
Accuracy: 0.93 +- 0.02
Precision: 0.91 +- 0.05
Recall: 0.91 +- 0.05
F1: 0.91 +- 0.03
ROCAUC: 0.93 +- 0.02


In [ ]:
nb1 = GaussianNB(var_smoothing=1e-9)
validate(nb1, 20)

Training results:
Accuracy: 0.94
Precision: 0.95
Recall: 0.89
F1: 0.92
ROCAUC: 0.93
Validation results:
Accuracy: 0.94
Precision: 0.95
Recall: 0.89
F1: 0.92
ROCAUC: 0.93
Test results:
Accuracy: 0.95 +- 0.03
Precision: 0.95 +- 0.03
Recall: 0.90 +- 0.05
F1: 0.92 +- 0.04
ROCAUC: 0.94 +- 0.03


In [ ]:
nb2 = GaussianNB(var_smoothing=1e-8)
validate(nb2, 20)

Training results:
Accuracy: 0.94
Precision: 0.96
Recall: 0.87
F1: 0.91
ROCAUC: 0.92
Validation results:
Accuracy: 0.94
Precision: 0.96
Recall: 0.87
F1: 0.91
ROCAUC: 0.93


In [ ]:
nb3 = GaussianNB(var_smoothing=1e-7)
validate(nb3, 20)

Training results:
Accuracy: 0.93
Precision: 0.97
Recall: 0.83
F1: 0.90
ROCAUC: 0.91
Validation results:
Accuracy: 0.94
Precision: 0.97
Recall: 0.86
F1: 0.91
ROCAUC: 0.92


In [ ]:
nb4 = GaussianNB(var_smoothing=1e-10)
validate(nb4, 20)

Training results:
Accuracy: 0.94
Precision: 0.94
Recall: 0.90
F1: 0.92
ROCAUC: 0.93
Validation results:
Accuracy: 0.94
Precision: 0.92
Recall: 0.91
F1: 0.91
ROCAUC: 0.93


In [ ]:
nb5 = GaussianNB(var_smoothing=1e-14)
validate(nb5, 20)

Training results:
Accuracy: 0.94
Precision: 0.94
Recall: 0.90
F1: 0.92
ROCAUC: 0.94
Validation results:
Accuracy: 0.93
Precision: 0.92
Recall: 0.90
F1: 0.91
ROCAUC: 0.93
Test results:
Accuracy: 0.93 +- 0.03
Precision: 0.93 +- 0.05
Recall: 0.89 +- 0.04
F1: 0.91 +- 0.04
ROCAUC: 0.93 +- 0.03
